In [1]:
# !pip install tensorflow

In [26]:
from PIL import Image
import glob

def resize_images(img_path):
	images=glob.glob(img_path + "/*.jpg")  
    
	print(len(images), " images to be resized.")

    # 파일마다 모두 224*224 사이즈로 바꾸어 저장합니다.
	target_size=(224,224)
	for img in images:
		old_img=Image.open(img)
		new_img=old_img.resize(target_size,Image.ANTIALIAS)
		new_img.save(img, "JPEG")
    
	print(len(images), " images resized.")

In [27]:
# 가위 이미지가 저장된 디렉토리 아래의 모든 jpg 파일을 읽어들여서
image_dir_path = r"moving\rockgame\dataset\scissors"
resize_images(image_dir_path)

print("가위 이미지 resize 완료!")

0  images to be resized.
0  images resized.
가위 이미지 resize 완료!


In [28]:
# 바위 이미지가 저장된 디렉토리 아래의 모든 jpg 파일을 읽어들여서
image_dir_path = r"moving\rockgame\dataset\rock"
resize_images(image_dir_path)

print("바위 이미지 resize 완료!")

0  images to be resized.
0  images resized.
바위 이미지 resize 완료!


In [29]:
# 보 이미지가 저장된 디렉토리 아래의 모든 jpg 파일을 읽어들여서
image_dir_path = r"moving\rockgame\dataset\paper"
resize_images(image_dir_path)

print("보 이미지 resize 완료!")

0  images to be resized.
0  images resized.
보 이미지 resize 완료!


In [30]:
import numpy as np
import os
import matplotlib.pyplot as plt
from PIL import Image

def load_data(img_path, number_of_data=3914):  # 가위바위보 이미지 개수 총합에 주의하세요.
    # 가위 : 0, 바위 : 1, 보 : 2
    img_size=224
    color=3
    #이미지 데이터와 라벨(가위 : 0, 바위 : 1, 보 : 2) 데이터를 담을 행렬(matrix) 영역을 생성합니다.
    imgs=np.zeros(number_of_data*img_size*img_size*color,dtype=np.int32).reshape(number_of_data,img_size,img_size,color)
    labels=np.zeros(number_of_data,dtype=np.int32)

    idx=0
    for file in glob.iglob(img_path+'/scissors/*.jpg'):
        img = Image.open(file).resize((224, 224))  # 크기를 (28, 28)로 변경
        img = np.array(img, dtype=np.int32)
        imgs[idx,:,:,:]=img    # 데이터 영역에 이미지 행렬을 복사
        labels[idx]=0   # 가위 : 0
        idx=idx+1

    for file in glob.iglob(img_path+'/rock/*.jpg'):
        img = Image.open(file).resize((224, 224))  # 크기를 (28, 28)로 변경
        img = np.array(img, dtype=np.int32)
        imgs[idx,:,:,:]=img    # 데이터 영역에 이미지 행렬을 복사
        labels[idx]=1   # 바위 : 1
        idx=idx+1  
    
    for file in glob.iglob(img_path+'/paper/*.jpg'):
        img = Image.open(file).resize((224, 224))  # 크기를 (28, 28)로 변경
        img = np.array(img, dtype=np.int32)
        imgs[idx,:,:,:]=img    # 데이터 영역에 이미지 행렬을 복사
        labels[idx]=2   # 보 : 2
        idx=idx+1
        
    print("학습데이터(x_train)의 이미지 개수는", idx,"입니다.")
    return imgs, labels

image_dir_path = "dataset" #폴더명
(x_train, y_train)=load_data(image_dir_path)
x_train_norm = x_train/255.0   # 입력은 0~1 사이의 값으로 정규화

print("x_train shape: {}".format(x_train.shape))
print("y_train shape: {}".format(y_train.shape))

학습데이터(x_train)의 이미지 개수는 3914 입니다.
x_train shape: (3914, 224, 224, 3)
y_train shape: (3914,)


In [31]:
# for i in range(100):
#     plt.imshow(x_train[i])
#     print('라벨 : ', y_train[i])
#     plt.show()

In [37]:
import tensorflow as tf
from tensorflow import keras
import numpy as np

model=keras.models.Sequential()
model.add(keras.layers.Conv2D(32, (3,3), activation='relu', input_shape=(224,224,3)))
model.add(keras.layers.MaxPool2D(2,2))
model.add(keras.layers.Conv2D(64, (3,3), activation='relu'))
model.add(keras.layers.BatchNormalization())
model.add(keras.layers.MaxPooling2D((2,2)))
model.add(keras.layers.Flatten())
model.add(keras.layers.Dense(128, activation='relu', kernel_regularizer=keras.regularizers.l2(0.01)))
model.add(keras.layers.Dropout(0.5))
model.add(keras.layers.Dense(20, activation='softmax'))

# model = keras.models.Sequential()
# model.add(keras.layers.Conv2D(64, (5,5), activation='relu', input_shape=(224,224,3)))  # 필터와 커널 크기 변경
# model.add(keras.layers.MaxPool2D(3,3))  # 풀링 크기 변경
# model.add(keras.layers.Conv2D(128, (3,3), activation='relu'))
# model.add(keras.layers.MaxPooling2D((2,2)))
# model.add(keras.layers.Flatten())
# model.add(keras.layers.Dense(256, activation='relu'))  # Dense 레이어 유닛 증가
# model.add(keras.layers.Dense(20, activation='softmax'))


model.summary()

Model: "sequential_5"

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━┓
┃ Layer (type)                    ┃ Output Shape           ┃       Param # ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━┩
│ conv2d_10 (Conv2D)              │ (None, 222, 222, 32)   │           896 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ max_pooling2d_10 (MaxPooling2D) │ (None, 111, 111, 32)   │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ conv2d_11 (Conv2D)              │ (None, 109, 109, 64)   │        18,496 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ batch_normalization             │ (None, 109, 109, 64)   │           256 │
│ (BatchNormalization)            │                        │               │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ max_pooling2d_11 (MaxPooling2D) │ (None, 54, 54, 64)     │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ flatten_5 (Flatten)             │ (None, 186624)         │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense_11 (Dense)                │ (None, 128)            │    23,888,000 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dropout_1 (Dropout)             │ (None, 128)            │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense_12 (Dense)                │ (None, 20)             │         2,580 │
└─────────────────────────────────┴────────────────────────┴───────────────┘

 Total params: 23,910,228 (91.21 MB)

 Trainable params: 23,910,100 (91.21 MB)

 Non-trainable params: 128 (512.00 B)

In [38]:
optimizer = keras.optimizers.Adam(learning_rate=0.0001)

model.compile(optimizer=optimizer,
             loss='sparse_categorical_crossentropy',
             metrics=['accuracy'])

model.fit(x_train_norm, y_train, epochs=10)

Epoch 1/10
123/123 ━━━━━━━━━━━━━━━━━━━━ 36s 284ms/step - accuracy: 0.7005 - loss: 3.2239
Epoch 2/10
123/123 ━━━━━━━━━━━━━━━━━━━━ 35s 285ms/step - accuracy: 0.9784 - loss: 1.0700
Epoch 3/10
123/123 ━━━━━━━━━━━━━━━━━━━━ 35s 286ms/step - accuracy: 0.9909 - loss: 0.7957
Epoch 4/10
123/123 ━━━━━━━━━━━━━━━━━━━━ 36s 289ms/step - accuracy: 0.9940 - loss: 0.6198
Epoch 5/10
123/123 ━━━━━━━━━━━━━━━━━━━━ 35s 287ms/step - accuracy: 0.9907 - loss: 0.5047
Epoch 6/10
123/123 ━━━━━━━━━━━━━━━━━━━━ 35s 281ms/step - accuracy: 0.9927 - loss: 0.4109
Epoch 7/10
123/123 ━━━━━━━━━━━━━━━━━━━━ 35s 286ms/step - accuracy: 0.9935 - loss: 0.3398
Epoch 8/10
123/123 ━━━━━━━━━━━━━━━━━━━━ 35s 285ms/step - accuracy: 0.9942 - loss: 0.2812
Epoch 9/10
123/123 ━━━━━━━━━━━━━━━━━━━━ 35s 285ms/step - accuracy: 0.9926 - loss: 0.2362
Epoch 10/10
123/123 ━━━━━━━━━━━━━━━━━━━━ 35s 286ms/step - accuracy: 0.9954 - loss: 0.1948


In [39]:
image_dir_path = "dataset/test"
(x_test, y_test)=load_data(image_dir_path,328)
x_test_norm = x_test/255.0

print("x_test shape : {}".format(x_test.shape))
print("y_test shape : {}".format(y_test.shape))

학습데이터(x_train)의 이미지 개수는 328 입니다.
x_test shape : (328, 224, 224, 3)
y_test shape : (328,)


In [40]:
test_loss, test_accuracy = model.evaluate(x_test_norm, y_test, verbose=2)
print("test_loss : {}".format(test_loss))
print("test_accuracy : {}".format(test_accuracy))

11/11 - 1s - 67ms/step - accuracy: 0.5305 - loss: 1.9970
test_loss : 1.996994972229004
test_accuracy : 0.5304877758026123


In [11]:
predicted_result = model.predict(x_test_norm)	# model이 추론한 확률값
predicted_labels = np.argmax(predicted_result, axis=1)

idx=100		# 값을 변경해서 찾아보자
print('model.predict() 결과 : ', predicted_result[idx])
print('model이 추론한 가장 가능성이 높은 결과 : ', predicted_labels[idx])
print('실제 데이터의 라벨 : ', y_test[idx])

11/11 ━━━━━━━━━━━━━━━━━━━━ 1s 47ms/step
model.predict() 결과 :  [9.9997389e-01 9.0234231e-08 2.6017997e-05 2.6286266e-31 3.9355646e-35
 3.5820798e-32 3.7356373e-36 2.7632927e-30 1.8339877e-36 1.9760968e-28
 9.4519344e-32 1.4290723e-31 7.6447136e-34 1.2683281e-35 1.4984826e-32
 1.3355235e-36 7.4142126e-32 2.1781345e-32 7.3302474e-37 2.8941114e-34]
model이 추론한 가장 가능성이 높은 결과 :  0
실제 데이터의 라벨 :  0


In [24]:
# plt.imshow(x_test[idx], cmap=plt.cm.binary)
# plt.show()

In [25]:
import random
wrong_predict_list=[]
for i, _ in enumerate(predicted_labels):
    if predicted_labels[i] != y_test[i]:
        wrong_predict_list.append(i)
        
samples = random.choices(population=wrong_predict_list, k=10) # k값을 수정하면 더 많은 값의 수를 볼 수 있다.

for n in samples:
    print("예측확률분포: " + str(predicted_result[n]))
    print("라벨: " + str(y_test[n]) + ", 예측결과: " + str(predicted_labels[n]))
    # plt.imshow(x_test[n], cmap=plt.cm.binary)
    print(n)
    # plt.show()

예측확률분포: [9.1581380e-01 8.4185921e-02 3.0026499e-07 5.3266351e-31 4.6002564e-33
 5.6007122e-32 3.3643070e-35 4.7751902e-28 6.2853316e-34 8.8028721e-29
 3.0363445e-31 1.3943476e-29 2.6320797e-32 3.2328453e-34 1.2807839e-31
 6.3428772e-33 3.2059319e-30 3.5811294e-31 8.7306473e-34 1.4664955e-30]
라벨: 2, 예측결과: 0
255
예측확률분포: [9.01450515e-01 1.69277638e-02 8.16217959e-02 5.65311717e-29
 3.17695451e-32 3.66394517e-30 2.90260779e-33 7.09590784e-27
 1.41013551e-33 1.92224202e-26 2.06463106e-29 5.33457016e-29
 6.81794247e-30 5.23593249e-33 3.37184324e-31 1.12692505e-32
 6.31249720e-30 1.12644972e-29 8.67842280e-33 2.92588960e-31]
라벨: 2, 예측결과: 0
321
예측확률분포: [9.46766317e-01 1.42178484e-04 5.30915298e-02 4.74291786e-29
 6.82116890e-33 3.78486010e-30 1.40443115e-33 2.43856184e-27
 4.65015877e-34 1.54397258e-26 1.78864033e-29 1.93927062e-29
 2.01290186e-30 2.34241678e-33 4.84295773e-31 9.73860886e-34
 4.04809635e-30 3.53323735e-30 1.14873220e-33 3.54525041e-32]
라벨: 2, 예측결과: 0
226
예측확률분포: [9.9794608e-01

In [13]:
model.save('dataset/model/model.keras', include_optimizer=False)